# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
weather_results_df = pd.read_csv('../WeatherPy/output_data/cities.csv').reset_index(drop=True)
weather_results_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# store lat and lng in locations
locations = weather_results_df[['Lat', 'Lng']]
humidity = weather_results_df['Humidity']
weather_results_df

In [ ]:
# plot heatmap
fig = gmaps.figure()

# create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity = max(humidity), point_radius=1)

# add layer
fig.add_layer(heat_layer)

fig


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Start narrowing down the results by weather conditions that I like
ideal_weather_df = (weather_results_df[weather_results_df['Max Temp']>24]).reset_index(drop=True)
ideal_weather_df = ideal_weather_df[ideal_weather_df['Wind Speed']<10]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Cloudiness']==0]
ideal_weather_df = ideal_weather_df.dropna(axis=1)
ideal_weather_df = ideal_weather_df.sort_values(by=['Humidity'], ascending=False)
if len(ideal_weather_df['City'])>10:
    ideal_weather_df = ideal_weather_df.head(10)
ideal_weather_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [ ]:
hotel_df = pd.DataFrame(ideal_weather_df).reset_index(drop=True)
hotel_df['Hotel Name'] = ""
hotel_df = hotel_df.reset_index()
hotel_df

In [ ]:
target_radius = 5000
target_type = "lodging"
    
# hotel_df['Lat, Lng'] = lat_lng
# for i in range(len(lat_lng)):
for index, row in hotel_df.iterrows():
    lat_lng = f"{hotel_df.iloc[index]['Lat']},{hotel_df.iloc[index]['Lng']}"
    params={
            "location": lat_lng,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

hotel_df.head()

In [ ]:
coordinates = list(zip(hotel_df['Lat'],hotel_df['Lng']))
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)
# Display Map
fig